## 到期收益率和即期利率相互转换

In [2]:
import numpy as np

In [3]:
def spot_to_par(
        self,
        Spot=None,#即期利率arrary
        Tenor=None,#期限arrary
        # Type="DISCRETE",
        ):
    assert len(self.Tenor)==len(self.Spot)
    principal_value=1/(1+self.Spot/100)**self.Tenor#不付息债，按照直接按照spot贴现
    interest=np.ceil(self.Tenor)-self.Tenor#应计利息
    # sum from 1 to n, 1/(1+st)^t
    def _coupon_df(_tenor):
        # 每个付息时间点
        # 一定要round，不然生成数列会有很低位的小数，和self.Tenor对不上
        # _tenor=self.Tenor[55]
        tenor_list=np.arange(_tenor,0,-1)[::-1].round(5)
        temp=np.isin(self.Tenor,tenor_list)
        spot_list=self.Spot[temp]/100
        output=np.sum(1/(1+spot_list)**tenor_list)#全价
        return output
    # 将self.Tenor升一个维度，方便用apply
    coupon_df=np.apply_along_axis(_coupon_df,axis=1,arr=self.Tenor.reshape(-1,1))

    # 算法更精确，目标是让spot曲线贴现的全价和par曲线贴现的全价直接相等，
    # 一般差异不大，0-10Y平均误差0.3BP，短端差异可达2BP
    # def func(c):
    #     return c*coupon_df+principal_value-(1+c)**interest
    # coupon=scipy.optimize.fsolve(func,np.empty(len(self.Tenor)))*100

    # 目标是让用spot贴现的净价始终等于1，净价=全价-t/365*c，应计利息不贴现
    coupon=(1-principal_value)/(coupon_df-interest)*100
    self.Par=coupon
    return self

In [186]:
def par_to_spot(
        Par=None,
        Tenor=None ):
    
    assert len(Par) == len(Tenor)
    Par = Par/100 
    Tenor = Tenor.round(5)#取近似避免无法匹配
    interest=(np.ceil(Tenor)-Tenor)*Par#应计利息
    discounted_factor = 1/(1+Par)**Tenor #不同spot下的折旧因子，把初始值先设为对应的到期收益率的折旧因子
    
    for i in range(len(Tenor)):
        if Tenor[i]<=1: #如果期限小于1，spot的折旧因子和到期收益率的折旧因子一致
            continue
        tenor_list=np.arange(Tenor[i]-1,0,-1)[::-1].round(5)#找到靴子的期限位置
        temp=np.isin(Tenor,tenor_list)
        before_period_flow = np.sum(discounted_factor[temp]*Par[i])#计算最后一期期之前的付息折现
        
        # 最终的折现因子=（全价-最后一期之前的付息的折现）/（1+c)
        discounted_factor[i] = (1+interest[i]- before_period_flow)/(1+Par[i])
    result = ((1/discounted_factor)**(1/Tenor)-1)*100 
    return(result)

In [188]:
Tenor=np.arange(0.1,10,0.1)
Par = Tenor*0.1+3
par_to_spot(Par = Par, Tenor = Tenor)

array([3.01      , 3.02      , 3.03      , 3.04      , 3.05      ,
       3.06      , 3.07      , 3.08      , 3.09      , 3.1       ,
       3.1062366 , 3.11387   , 3.12259676, 3.13220047, 3.14252274,
       3.15344503, 3.16487697, 3.17674854, 3.18900468, 3.20160159,
       3.20973023, 3.21847976, 3.22778045, 3.23757437, 3.24781307,
       3.25845572, 3.2694677 , 3.28081953, 3.29248596, 3.30444525,
       3.31333358, 3.32262473, 3.33229037, 3.34230561, 3.35264857,
       3.36329992, 3.37424255, 3.38546131, 3.39694271, 3.40867477,
       3.4180301 , 3.4276898 , 3.43763932, 3.44786558, 3.45835676,
       3.46910217, 3.48009216, 3.491318  , 3.50277177, 3.51444631,
       3.52415818, 3.5341197 , 3.54432249, 3.5547589 , 3.56542191,
       3.57630514, 3.58740273, 3.59870932, 3.61022004, 3.62193042,
       3.63195216, 3.64218953, 3.6526374 , 3.66329103, 3.6741461 ,
       3.68519861, 3.69644489, 3.70788161, 3.71950568, 3.73131432,
       3.74162823, 3.75213519, 3.76283198, 3.77371563, 3.78478

# 远期利率和即期利率之间的转换

零票息折现因子为  
$p(0,T) = E[exp(-\int_{0}^{T}r(s)ds)] = exp(-TR(0,T)) = exp(-\int_{0}^{T}f(0,s)ds)$

以上公式是零票息折现因子的表示,$p(0,T)$中的0表示的是现在时间点，T表示的是这是T期以后的现金流的折现因子  
因为上面公式$r(s)$和$f(0,s)$分别是短期利率的瞬时值和远期利率的瞬时值，也就是说用的是无穷小时间段的连续复利模型来计算的，所以自然得到的形式就是e指数，同时连续复利的乘积就变成e指数上的求和，极限情况下就是积分。$R(0,T)$完全是一个定义问题，定义即期利率乘以T等于第一个表达式中的形式。  


所以 $R(0,T) = \frac{1}{T}\int_{0}^{T}f(0,s)ds$       $ R(0,T) + T\frac{\partial R(0,T) }{\partial T} = f(0,T) $


